# GeoTagger - Spacy

In [1]:
# Import necessary libraries.
import re, warnings, urllib, requests, spacy, geopy, folium, os, sys, glob
import pandas as pd
import numpy as np
from collections import Counter
from geopy.extra.rate_limiter import RateLimiter

# Import project-specific functions. 
# Python files (.py) have to be in same folder to work.
lib_path = os.path.abspath(os.path.join(os.path.dirname('Correspondence_XML_parser.py'), '../Scripts'))
sys.path.append(lib_path)

from Correspondence_XML_parser import *

nlp = spacy.load('en_core_web_sm')

# Ignore warnings related to deprecated functions.
warnings.filterwarnings('ignore')

## Get XML Files

In [2]:
%%time

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/Data"
files = glob.glob(abs_dir + "/PSC/Richards/ESR-XML-Files-MHS/*.xml")

len(files)

CPU times: user 808 µs, sys: 1.26 ms, total: 2.07 ms
Wall time: 1.94 ms


20

## Build Dataframe

In [3]:
%%time

# Build dataframe from XML files.
# build_dataframe() called from Correspondence_XML_parser
df = build_dataframe(files)

df.head(3)

/Users/quinn.wi/Documents/Data/PSC/Richards/ESR-XML-Files-MHS/ESR-EDA-1893-09-24.xml 

CPU times: user 10.2 ms, sys: 3.59 ms, total: 13.8 ms
Wall time: 15.7 ms


,file,date,source,target,subjects,references,text
0,ESR-EDA-1892-01-08.xml,1892-01-08,richards-ellen,atkinson-edward,"1893 Chicago World's Fair,Aladdin Oven,New Eng...","palmer-bertha,hovey-e,daniells-unknown",Boston Jan 8 1892 My dear Mr Atkinson I enclo...
1,ESR-EDA-1892-04-12.xml,1892-04-12,richards-ellen,atkinson-edward,"Aladdin Oven,nutrition,cooking",abel-mary,April 12— Dear Mr Atkinson I expect Mrs Abel ...
2,ESR-EDA-1892-04-07.xml,1892-04-07,richards-ellen,atkinson-edward,"Aladdin Oven,Nutrition,cooking","conro-emma,abel-mary","Boston, April 7, 1892 My dear Mr. Atkinson I ..."


## Get Place Names

In [4]:
%%time

def get_placenames(text):
    doc = nlp(text)
    places = [ent.text for ent in doc.ents if ent.label_ in ['LOC', 'GPE']]
    return places
    
df['places'] = df['text'].apply(lambda x: get_placenames(x))

df = df[['file', 'date', 'places']]
df = df.explode('places')

df.head(3)

CPU times: user 522 ms, sys: 9.81 ms, total: 532 ms
Wall time: 534 ms


,file,date,places
0,ESR-EDA-1892-01-08.xml,1892-01-08,Boston
0,ESR-EDA-1892-01-08.xml,1892-01-08,Chicago
1,ESR-EDA-1892-04-12.xml,1892-04-12,Chicago


## GeoCode Places

In [5]:
%%time

# https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/
def geocode(place):
#     url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(place) +'?format=json'
    url = 'https://nominatim.openstreetmap.org/search/' + str(place) + '?format=json'
    response = requests.get(url).json()
    if (len(response) != 0):
#         Default (response[0]): select first search hit in OpenStreetMap.
        return (float(response[0]['lat']), float(response[0]['lon']))
    else:
        return None 

df['coordinates'] = df['places'].apply(geocode)
df[['lat', 'lon']] = pd.DataFrame(df['coordinates'].tolist(), index = df.index)

# Convert to floats.
df['lat'] = df['lat'].apply(lambda x: float(x))
df['lon'] = df['lon'].apply(lambda x: float(x))

df = df.dropna()

df.head(3)

CPU times: user 403 ms, sys: 31.6 ms, total: 435 ms
Wall time: 19.4 s


,file,date,places,coordinates,lat,lon
0,ESR-EDA-1892-01-08.xml,1892-01-08,Boston,"(42.3602534, -71.0582912)",42.360253,-71.058291
0,ESR-EDA-1892-01-08.xml,1892-01-08,Chicago,"(41.8755616, -87.6244212)",41.875562,-87.624421
1,ESR-EDA-1892-04-12.xml,1892-04-12,Chicago,"(41.8755616, -87.6244212)",41.875562,-87.624421


### Save Results

In [6]:
%%time

df.to_csv(os.path.abspath('../../lab_space/projects/richards/geo/data/richards_geoReferences.csv'), 
                          sep = ',', index = False)

CPU times: user 2.53 ms, sys: 1.58 ms, total: 4.11 ms
Wall time: 3.05 ms
